# Batch pre-computed recommendations

# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import os
import sys

import redis
from dotenv import load_dotenv
from loguru import logger
from pydantic import BaseModel
from tqdm.auto import tqdm

load_dotenv()

sys.path.insert(0, "..")

# Controller

In [3]:
class Args(BaseModel):
    testing: bool = False
    run_name: str = "003-alias-champion"
    notebook_persist_dp: str = None
    random_seed: int = 41

    redis_host: str = "localhost"
    redis_port: int = 6379
    redis_key_prefix: str = "output:i2i:"

    batch_recs_fp: str = "data/003-alias-champion/batch_recs.jsonl"

    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")
        os.makedirs(self.notebook_persist_dp, exist_ok=True)
        self.batch_recs_fp = f"{self.notebook_persist_dp}/batch_recs.jsonl"

        return self


args = Args().init()

print(args.model_dump_json(indent=2))

{
  "testing": false,
  "run_name": "003-alias-champion",
  "notebook_persist_dp": "/Users/dvq/frostmourne/fsds/fsds-recsys/chapters/l7/notebooks/data/003-alias-champion",
  "random_seed": 41,
  "redis_host": "localhost",
  "redis_port": 6379,
  "redis_key_prefix": "output:i2i:",
  "batch_recs_fp": "/Users/dvq/frostmourne/fsds/fsds-recsys/chapters/l7/notebooks/data/003-alias-champion/batch_recs.jsonl"
}


# Load batch recs into Redis

In [4]:
r = redis.Redis(host=args.redis_host, port=args.redis_port, db=0, decode_responses=True)
assert (
    r.ping()
), f"Redis at {args.redis_host}:{args.port} is not running, please make sure you have started the Redis docker service"

In [5]:
def store_recommendations(file_path: str):
    with open(file_path, "r") as file:
        for line in tqdm(file):
            rec_data = json.loads(line)
            target_item = rec_data["target_item"]
            key = args.redis_key_prefix + target_item
            r.set(
                key,
                json.dumps(
                    {
                        "rec_item_ids": rec_data["rec_item_ids"],
                        "rec_scores": rec_data["rec_scores"],
                    }
                ),
            )


def get_recommendations(target_item):
    key = args.redis_key_prefix + target_item
    rec_data = r.get(target_item)
    if rec_data:
        return json.loads(rec_data)
    return None


def get_example_keys(count=5):
    keys = r.scan_iter(match=args.redis_key_prefix + "*", count=count)
    output = []
    for i, key in enumerate(keys, 1):
        output.append(key)
        if i >= count:
            return output

In [6]:
logger.info(f"Loading batch recs output from {args.batch_recs_fp}...")
store_recommendations(args.batch_recs_fp)

2024-10-04 16:38:17.765 | INFO     | __main__:<module>:1 - Loading batch recs output from /Users/dvq/frostmourne/fsds/fsds-recsys/chapters/l7/notebooks/data/003-alias-champion/batch_recs.jsonl...


0it [00:00, ?it/s]

In [7]:
get_recommendations(get_example_keys()[0])

{'rec_item_ids': ['B001QRJ7VY',
  'B0039LME9W',
  'B00DOD51UY',
  'B000SFK0SO',
  'B001E90A60',
  'B002I0J8RQ',
  'B002D2Y3IS',
  'B004WLRR4K',
  'B0013B30SY',
  'B00KX766ZS',
  'B0055VAB30',
  'B001EYUQY6',
  'B0058T3492',
  'B01FSKACPY',
  'B002VAWXO8',
  'B002SUTB28',
  'B004IYY8PW',
  'B000R4NFC4',
  'B001EYUOHK',
  'B006BZ1PCA',
  'B00006FWUU',
  'B0009SQFHK',
  'B00002ST6O',
  'B003RS1AIS',
  'B014XCWZA8',
  'B003LPUBHS',
  'B002JTX87C',
  'B008U2XJNA',
  'B00KUYTOLM',
  'B002BS47L2',
  'B07X54548H',
  'B00439DIZ8',
  'B001EYUX4O',
  'B001EYUXZI',
  'B0055CPCJ2',
  'B004TMGZ72',
  'B00NBHUWN4',
  'B007VYW3US',
  'B01GOK2GQU',
  'B001KX504C',
  'B003QCJLQI',
  'B002G9U72Y',
  'B003ANMB6A',
  'B001EYUT6G',
  'B005UUO9CK',
  'B00D6LPE2E',
  'B07WYX8SKK',
  'B07YNKBR7L',
  'B00AYABWYE',
  'B004HILZX2',
  'B0050SYXSW',
  'B077SN7QN3',
  'B00009KO3S',
  'B009AR2CAU',
  'B083GM38LQ',
  'B01I14IASK',
  'B01KBL4ISW',
  'B07NKFR9M4',
  'B001ELJEAQ',
  'B002JTX9XK',
  'B09M6YFK7Y',
  'B000A